In [1]:
from huggingface_hub import login
import inspect
import dotenv
import os

dotenv.load_dotenv()
hf_token = os.getenv("HUGGINGFACE_TOKEN")
login(hf_token)

/root/miniconda3/envs/heloc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    # device_map="auto",
    # torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
)
device = torch.device("cuda:0")
# model.to(device)

# input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# outputs = model.generate(**input_ids, max_new_tokens=32)
# print(tokenizer.decode(outputs[0]))


`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:50<00:00, 12.57s/it]
The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<bos>Write me a poem about Machine Learning.

In silicon valleys, where data flows,
A new intelligence, silently grows.
Machine Learning, a name whispered low,
Algorithms dance, a digital


In [11]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(inputs["input_ids"], max_length=3000, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

SHAP, LIME 을 Gemma가 알고 있는지 체크

In [4]:
# prompt = "What is SHAP and LIME? Plase explain with detail."

In [5]:
# gemma_response = generate_response(prompt)
# print(f"Gemma: {gemma_response}")

잘 알고 있다!

2-1.

Test natural language without format

In [9]:
prompt = inspect.cleandoc('''
Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please respond as if you were a human, using natural conversational tone. Be engaging, empathetic, and use phrases and expressions that sound like they’re coming from a real person, keeping the tone friendly and conversational. Avoid sounding overly formal or robotic.
Please provide a response without explicitly using terms like 'model,' 'probability,' or directly mentioning numbers. Instead, explain the concepts in simple, intuitive language that avoids technical jargon.
At the end of the response, provide a personalized piece of advice for the user on how they can improve or maintain their risk performance in the future.

Please follow these steps to explain the prediction:

1. SHAP Analysis: Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
2. LIME Analysis: Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
3. Insight Synthesis: Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the final prediction.
4. Final Explanation for Non-Experts: Instead of focusing on technical jargon, ensure that the final explanation is understandable to non-experts in machine learning or finance.

Context:
1. Prediction Probability
- Good: 0.57029474
- Bad: 0.4297053
- Predicted to Good

2. SHAP analysis (Feature, SHAP Importance)
- (NumSatisfactoryTrades, 0.320243)
- (PercentTradesNeverDelq, 0.305187)
- (MSinceMostRecentInqexcl7days, 0.293064)
- (NumTradesOpeninLast12M, 0.227222)
- (MaxDelq2PublicRecLast12M, 0.212770)
- (NumBank2NatlTradesWHighUtilization, 0.073882)
- (NumTotalTrades, 0.065301)
- (NumRevolvingTradesWBalance, 0.049232)
- (MSinceMostRecentDelq, 0.042432)
- (NumInqLast6M, 0.039852)

3. LIME analysis (Feature, LIME Importance)
- (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
- (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
- (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
- (NumSatisfactoryTrades > 27.00, -0.110831)
- (NumTradesOpeninLast12M > 3.00,  0.058771)
- (NumRevolvingTradesWBalance > 5.00, 0.036777)
- (NumTotalTrades > 29.00, -0.027887)
- (PercentInstallTrades <= 20.00, 0.027751)
- (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
- (49.50 < PercentTradesWBalance <= 67.00, -0.013193)

Answer:
1. SHAP Analysis: First, explain each feature's SHAP importance and how it contributes to the final prediction.
2. LIME Analysis: Then, explain the individual feature importance from LIME and its role in the prediction.
3. Conclusion: Finally, synthesize the insights from both SHAP and LIME to provide a comprehensive, easy-to-understand conclusion.
''')

In [10]:
gemma_response = generate_response(prompt)
print(f"Gemma: {gemma_response}")

Gemma: Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please respond as if you were a human, using natural conversational tone. Be engaging, empathetic, and use phrases and expressions that sound like they’re coming from a real person, keeping the tone friendly and conversational. Avoid sounding overly formal or robotic.
Please provide a response without explicitly using terms like 'model,' 'probability,' or directly mentioning numbers. Instead, explain the concepts in simple, intuitive language that avoids technical jargon.
At the end of the response, provide a personalized piece of advice fo

2-2.

In [12]:
prompt = inspect.cleandoc('''
Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please respond as if you were a human, using natural conversational tone. Be engaging, empathetic, and use phrases and expressions that sound like they’re coming from a real person, keeping the tone friendly and conversational. Avoid sounding overly formal or robotic.
Please provide a response without explicitly using terms like 'model,' 'probability,' or directly mentioning numbers. Instead, explain the concepts in simple, intuitive language that avoids technical jargon.
At the end of the response, provide a personalized piece of advice for the user on how they can improve or maintain their risk performance in the future.

Before answering, please think steyp by step coincisely in these steps to explain the prediction.
1. SHAP Analysis: Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
This step should be inside <SHAP>$$INSERT TEXT HERE$$</SHAP> tag.
2. LIME Analysis: Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
This step should be inside <LIME>$$INSERT TEXT HERE$$</LIME> tag.
3. Insight Synthesis: Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the final prediction.
This step should be inside <Insight>$$INSERT TEXT HERE$$</Insight> tag.
4. Final Explanation for Non-Experts: Instead of focusing on technical jargon, ensure that the final explanation is understandable to non-experts in machine learning or finance.
This step should be inside <Conclusion>$$INSERT TEXT HERE$$</Conclusion> tag.

Context:
1. Prediction Probability
- Good: 0.57029474
- Bad: 0.4297053
- Predicted to Good

2. SHAP analysis (Feature, SHAP Importance)
- (NumSatisfactoryTrades, 0.320243)
- (PercentTradesNeverDelq, 0.305187)
- (MSinceMostRecentInqexcl7days, 0.293064)
- (NumTradesOpeninLast12M, 0.227222)
- (MaxDelq2PublicRecLast12M, 0.212770)
- (NumBank2NatlTradesWHighUtilization, 0.073882)
- (NumTotalTrades, 0.065301)
- (NumRevolvingTradesWBalance, 0.049232)
- (MSinceMostRecentDelq, 0.042432)
- (NumInqLast6M, 0.039852)

3. LIME analysis (Feature, LIME Importance)
- (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
- (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
- (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
- (NumSatisfactoryTrades > 27.00, -0.110831)
- (NumTradesOpeninLast12M > 3.00,  0.058771)
- (NumRevolvingTradesWBalance > 5.00, 0.036777)
- (NumTotalTrades > 29.00, -0.027887)
- (PercentInstallTrades <= 20.00, 0.027751)
- (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
- (49.50 < PercentTradesWBalance <= 67.00, -0.013193)

Answer:
''')

In [13]:
gemma_response = generate_response(prompt)
print(f"Gemma: {gemma_response}")

Gemma: Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please respond as if you were a human, using natural conversational tone. Be engaging, empathetic, and use phrases and expressions that sound like they’re coming from a real person, keeping the tone friendly and conversational. Avoid sounding overly formal or robotic.
Please provide a response without explicitly using terms like 'model,' 'probability,' or directly mentioning numbers. Instead, explain the concepts in simple, intuitive language that avoids technical jargon.
At the end of the response, provide a personalized piece of advice fo

2.3

In [ ]:
prompt = inspect.cleandoc('''
Question:
The following is the result of binary classification using the HELOC (Home Equity Line of Credit) Dataset and XGBClassifier to classify RiskPerformance into “Good” and “Bad.”
The value “Bad” indicates that a consumer was 90 days past due or worse at least once over a period of 24 months from when the credit account was opened. The value “Good” indicates that they have made their payments without ever being more than 90 days overdue.
Please respond as if you were a human, using natural conversational tone. Be engaging, empathetic, and use phrases and expressions that sound like they’re coming from a real person, keeping the tone friendly and conversational. Avoid sounding overly formal or robotic.
Please provide a response without explicitly using terms like 'model,' 'probability,' or directly mentioning numbers. Instead, explain the concepts in simple, intuitive language that avoids technical jargon.
At the end of the response, provide a personalized piece of advice for the user on how they can improve or maintain their risk performance in the future.

Before answering, please think steyp by step coincisely in these steps to explain the prediction.
1. SHAP Analysis: Analyze the key features from the SHAP analysis, explaining how each feature contributes to the prediction.
This step should be inside <SHAP>$$INSERT TEXT HERE$$</SHAP> tag.
2. LIME Analysis: Analyze the key features from the LIME analysis, explaining the contribution of each feature in terms of how it influences the prediction.
This step should be inside <LIME>$$INSERT TEXT HERE$$</LIME> tag.
3. Insight Synthesis: Based on the individual feature analyses from SHAP and LIME, synthesize the insights to provide a comprehensive conclusion. The conclusion should focus on how these features work together to influence the final prediction.
This step should be inside <Insight>$$INSERT TEXT HERE$$</Insight> tag.
4. Final Explanation for Non-Experts: Provide the prediction result and explain the comprehensive reasoning behind the result, considering multiple factors that contributed to this outcome. Ensure the explanation is clear, detailed, and avoids using technical terms or direct references to probabilities or numbers, so that the final explanation is understandable to non-experts in machine learning or finance.
This step should be inside <Conclusion>$$INSERT TEXT HERE$$</Conclusion> tag.

Context:
1. Prediction Probability
- Good: 0.57029474
- Bad: 0.4297053
- Predicted to Good

2. SHAP analysis (Feature, SHAP Importance)
- (NumSatisfactoryTrades, 0.320243)
- (PercentTradesNeverDelq, 0.305187)
- (MSinceMostRecentInqexcl7days, 0.293064)
- (NumTradesOpeninLast12M, 0.227222)
- (MaxDelq2PublicRecLast12M, 0.212770)
- (NumBank2NatlTradesWHighUtilization, 0.073882)
- (NumTotalTrades, 0.065301)
- (NumRevolvingTradesWBalance, 0.049232)
- (MSinceMostRecentDelq, 0.042432)
- (NumInqLast6M, 0.039852)

3. LIME analysis (Feature, LIME Importance)
- (MSinceMostRecentInqexcl7days <= -7.00, -0.186905)
- (6.00 < MaxDelq2PublicRecLast12M <= 7.00, -0.135623)
- (96.00 < PercentTradesNeverDelq <= 100.00, -0.124091)
- (NumSatisfactoryTrades > 27.00, -0.110831)
- (NumTradesOpeninLast12M > 3.00,  0.058771)
- (NumRevolvingTradesWBalance > 5.00, 0.036777)
- (NumTotalTrades > 29.00, -0.027887)
- (PercentInstallTrades <= 20.00, 0.027751)
- (NumTrades90Ever2DerogPubRec <= 0.00, -0.019881)
- (49.50 < PercentTradesWBalance <= 67.00, -0.013193)

Answer:
''')